# **Meses de oferta (Month of Supply)**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "puebla" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido','Estacionamientos':'estacionamiento'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['año','mes']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
# Concatenar los DataFrames
fin_pachuca = pd.concat(dataframes_list)

# Filtrar registros con año diferente de 0
fin_pachuca_filtrado = fin_pachuca[fin_pachuca['año'] != 0]

# Contar los registros por año y mes
conteo_registros_por_mes = fin_pachuca_filtrado.groupby(['año', 'mes']).size().reset_index(name='conteo')
conteo_registros_por_mes

,año,mes,conteo
0,2019,1,259
1,2019,2,250
2,2019,3,276
3,2019,4,258
4,2019,5,302
...,...,...,...
58,2023,11,902
59,2023,12,1096
60,2024,1,740
61,2024,2,865


In [5]:
conteos_por_año = fin_pachuca.groupby('año').size()

# Filtrar el conteo para cada año específico
conteo_2022 = conteos_por_año.get(2022, 0)  # Si no hay registros para el año 2022, devolverá 0
conteo_2023 = conteos_por_año.get(2023, 0)  # Si no hay registros para el año 2023, devolverá 0
conteo_2024 = conteos_por_año.get(2024, 0)  # Si no hay registros para el año 2024, devolverá 0


print("Conteo de registros para 2022:", conteo_2022)
print("Conteo de registros para 2023:", conteo_2023)
print("Conteo de registros para 2023:", conteo_2024)

Conteo de registros para 2022: 3104
Conteo de registros para 2023: 10837
Conteo de registros para 2023: 2497


In [6]:
promedio_2022 = (conteo_2022/12).round(2)
promedio_2023 = (conteo_2023/12).round(2)
promedio_2024 = (conteo_2024/12).round(2)
print("Promedio de créditos en 2022:", promedio_2022)
print("Promedio de créditos en 2023:", promedio_2023)
print("Promedio de créditos en 2024:", promedio_2024)

Promedio de créditos en 2022: 258.67
Promedio de créditos en 2023: 903.08
Promedio de créditos en 2024: 208.08


In [7]:
# Dividir los conteos por los promedios respectivos de cada año
#conteo_registros_por_mes['promedio_creditos'] = conteo_registros_por_mes.apply(lambda row: round(row['conteo'] / promedio_2022, 2) if row['año'] == 2022 else round(row['conteo'] / promedio_2023, 2), axis=1)
#conteo_registros_por_mes

# Dividir los conteos por los promedios respectivos de cada año
conteo_registros_por_mes['promedio_creditos'] = conteo_registros_por_mes.apply(
    lambda row: round(row['conteo'] / promedio_2022, 2) if row['año'] == 2022 else
                round(row['conteo'] / promedio_2023, 2) if row['año'] == 2023 else
                round(row['conteo'] / promedio_2024, 2) if row['año'] == 2024 else
                round(row['conteo'] / promedio_2023, 2),  # Fallback para otros años
    axis=1
)
conteo_registros_por_mes

,año,mes,conteo,promedio_creditos
0,2019,1,259,0.29
1,2019,2,250,0.28
2,2019,3,276,0.31
3,2019,4,258,0.29
4,2019,5,302,0.33
...,...,...,...,...
58,2023,11,902,1.00
59,2023,12,1096,1.21
60,2024,1,740,3.56
61,2024,2,865,4.16


In [8]:
# Convertir los valores de la columna mes a enteros
conteo_registros_por_mes['mes'] = conteo_registros_por_mes['mes'].astype(int)
# Mapear los números de mes a sus nombres correspondientes
meses_texto = {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}
# Convertir el número de mes en texto
conteo_registros_por_mes['mes'] = conteo_registros_por_mes.apply(lambda row: f"{meses_texto[row['mes']]}_{int(row['año'])}", axis=1)
# Eliminar la columna 'año' ahora que ya no es necesaria
conteo_registros_por_mes = conteo_registros_por_mes.drop(columns=['año'])
conteo_registros_por_mes

,mes,conteo,promedio_creditos
0,Enero_2019,259,0.29
1,Febrero_2019,250,0.28
2,Marzo_2019,276,0.31
3,Abril_2019,258,0.29
4,Mayo_2019,302,0.33
...,...,...,...
58,Noviembre_2023,902,1.00
59,Diciembre_2023,1096,1.21
60,Enero_2024,740,3.56
61,Febrero_2024,865,4.16


In [12]:
# Filtrar solo los registros correspondientes a 2023
conteo_2023 = conteo_registros_por_mes[conteo_registros_por_mes['mes'].str.contains('2023')]
# Seleccionar las columnas mes, conteo y promedio_creditos
conteo_2023 = conteo_2023[['mes', 'conteo', 'promedio_creditos']]
conteo_2023

,mes,conteo,promedio_creditos
48,Enero_2023,647,0.72
49,Febrero_2023,863,0.96
50,Marzo_2023,931,1.03
51,Abril_2023,806,0.89
52,Mayo_2023,948,1.05
53,Junio_2023,986,1.09
54,Julio_2023,840,0.93
55,Agosto_2023,880,0.97
56,Septiembre_2023,898,0.99
57,Octubre_2023,1040,1.15


In [13]:
# Obtener los datos
promedio = conteo_2023['promedio_creditos']
meses = conteo_2023['mes']

# Crear figura
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=meses,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for mes, dia in zip(meses, promedio):
    fig.add_annotation(
        x=mes,
        y=dia,
        text=str(dia),  # Texto que se mostrará (el valor del día)
        showarrow=False,  # No mostrar la flecha
        font=dict(color='black', size=12),  # Estilo del texto
        xshift=0,  # Desplazamiento horizontal del texto
        yshift=17,  # Desplazamiento vertical del texto
        textangle=0  # Ángulo del texto
    )
# Actualizar diseño
fig.update_layout(
    #title='Meses de oferta',
    xaxis_title='Mes de oferta créditos',
    yaxis_title='Promedio',
    xaxis=dict(gridcolor='#dddcda', tickmode='array', tickvals=list(range(len(meses))), ticktext=meses),
    yaxis=dict(gridcolor='#dddcda', gridwidth=1),
    plot_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)   
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_scatt_MonthSupply', carpeta='assets/graficas')
fig.show()